In [1]:
import requests
import re
from bs4 import BeautifulSoup
import math
import pandas as pd
import time
import random
import wrangle

https://www.scrapingdog.com/blog/scrape-linkedin-jobs/

https://www.geeksforgeeks.org/using-jupyter-notebook-in-virtual-environment/

# Target URL
Follow the steps to produce a target URL:
1. Goto Linkedin and search for a job with any filters you want.
2. Open up the Developer inspector (Use Google Chrome) and scroll down until the next set of jobs load.
3. In the network tab there should be a "search" request when it loads the new jobs.
4. Grab that URL in the search request.
5. replace the last NUMBER in the URL with a {} in order to utilize a .format() for looping later. ie 'start=0' -> 'start={}'

In [2]:
target_url = 'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Data%2BScientist&location=United%2BStates&geoId=103644278&f_TPR=r86400&f_JT=F&f_E=2%2C4&f_WT=2&currentJobId=3692555868&position=2&pageNum=2&start={}'

# Job ID Loop
Used to get the Job ID to appened to the job URL for crawling

In [3]:
job_ids_list = wrangle.job_id_scrape(target_url, 1000)

In [4]:
len(job_ids_list)

992

# Job Loop
This is made outside of a function on purpose because if the function fails or is interupted at any step it won't return a DF.

In [5]:
job_data = {}
for i in range(0, len(job_ids_list)):
    job_data, job_ids_list = wrangle.job_scrape(job_data, job_ids_list)

KeyboardInterrupt: 

In [6]:
df = pd.DataFrame(job_data)
df.head()

,job_title,company,job_info,when_posted,num_applicants,seniority,employment_type,job_function,industry,job_url
0,Data Scientist Program Manager,Studio X,General OverviewStudio X is a global innovatio...,20 hours ago,Be among the first 25 applicants,Entry level,Full-time,Engineering and Information Technology ...,"Technology, Information and Internet",https://www.linkedin.com/jobs-guest/jobs/api/j...
1,"SENIOR MACHINE LEARNING ENGINEER - AUSTIN,TX O...",Harnham,"SENIOR MACHINE LEARNING ENGINEERAUSTIN, TX - O...",1 hour ago,Be among the first 25 applicants,Mid-Senior level,Full-time,Engineering and Information Technology ...,Staffing and Recruiting,https://www.linkedin.com/jobs-guest/jobs/api/j...
2,Research Engineer (Computer Vision) - Remote,Talentify.io,Joining our team at Hiring Company offers you ...,3 hours ago,34 applicants,Entry level,Full-time,Research,Staffing and Recruiting,https://www.linkedin.com/jobs-guest/jobs/api/j...
3,Principal Data Scientist,Harnham,Principal Data Scientist (Remote)United States...,1 hour ago,Be among the first 25 applicants,Entry level,Full-time,Engineering and Information Technology ...,Staffing and Recruiting,https://www.linkedin.com/jobs-guest/jobs/api/j...
4,Deep Learning Data Scientist,Harnham,"Biotech Research | Boston, MA | Remote$160,000...",1 hour ago,Be among the first 25 applicants,Entry level,Full-time,Engineering and Information Technology ...,Staffing and Recruiting,https://www.linkedin.com/jobs-guest/jobs/api/j...
